In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,8,3,padding = 1) #28x28
        self.batch1 = nn.BatchNorm2d(num_features=8)
        self.pool1 = nn.MaxPool2d(2,2) #14x14
        
        
        self.conv2 = nn.Conv2d(8,16,3) #12x12
        self.batch2 = nn.BatchNorm2d(num_features=16)
        self.conv3  = nn.Conv2d(16,32,3) #10x10
        self.batch3 = nn.BatchNorm2d(num_features=32)
        self.pool2 = nn.MaxPool2d(2,2) #5x5
             
        self.conv4 = nn.Conv2d(32,32,3)#3x3 
        self.conv6 = nn.Conv2d(32,10,3)#1

    def forward(self, x):
        
        x = self.pool1(self.batch1(F.relu(self.conv1(x))))
        x = self.pool2(self.batch3(F.relu(self.conv3(self.batch2(F.relu(self.conv2(x)))) )))
        x = F.relu(self.conv4(x))
        x = self.conv6(x)
#         print (x.shape)
        x = x.view(-1,10)
#         print(x.shape)      
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         MaxPool2d-3            [-1, 8, 14, 14]               0
            Conv2d-4           [-1, 16, 12, 12]           1,168
       BatchNorm2d-5           [-1, 16, 12, 12]              32
            Conv2d-6           [-1, 32, 10, 10]           4,640
       BatchNorm2d-7           [-1, 32, 10, 10]              64
         MaxPool2d-8             [-1, 32, 5, 5]               0
            Conv2d-9             [-1, 32, 3, 3]           9,248
           Conv2d-10             [-1, 10, 1, 1]           2,890
Total params: 18,138
Trainable params: 18,138
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.20
Params size (MB): 0.07
Estimated Tot

/home/jd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:
torch.manual_seed(1)
batch_size = 256

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
l = []
acc = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    l.append(test_loss)
    acc.append(100. * correct / len(test_loader.dataset))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0375, momentum=0.9)

for epoch in range(1, 21):
    print (epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/235 [00:00<?, ?it/s]

1


/home/jd/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.0508, Accuracy: 9832/10000 (98%)

2


  0%|          | 0/235 [00:00<?, ?it/s]

...

  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.0324, Accuracy: 9915/10000 (99%)

15


  0%|          | 0/235 [00:00<?, ?it/s]


Test set: Average loss: 0.0300, Accuracy: 9919/10000 (99%)

16


loss=0.0003290697932243347 batch_id=63:  27%|██▋       | 64/235 [00:02<00:06, 27.54it/s] 

###Final accuracy i got = 99.44 at epoch 12 